In [2]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import pickle
import time

In [5]:
driver = webdriver.Chrome()

In [6]:
driver.get('https://chartmetric.io/login')

In [7]:
time.sleep(5)

In [8]:
pickle_pw = pickle.load(open("chartmetric_pw.pickle", "rb"))

In [9]:
driver.find_element_by_css_selector( "body > div > div:nth-child(2) > div.container.ng-scope > div > div > div > div > form > div:nth-child(4) > div > input" ).send_keys( "lucaseo1991@gmail.com" )
driver.find_element_by_css_selector( "body > div > div:nth-child(2) > div.container.ng-scope > div > div > div > div > form > div:nth-child(5) > div > input" ).send_keys( pickle_pw )

In [10]:
driver.find_element_by_css_selector("body > div > div:nth-child(2) > div.container.ng-scope > div > div > div > div > form > div:nth-child(7) > div > button").click()

In [11]:
driver.get('https://chartmetric.io/search')

In [12]:
time.sleep(5)

In [40]:
artist = "Cardi B"

In [41]:
driver.find_element_by_css_selector( "body > div:nth-child(1) > div:nth-child(2) > div > div > form > input" ).send_keys(artist)

In [42]:
time.sleep(5)

In [43]:
from selenium.webdriver.common.keys import Keys
driver.find_element_by_css_selector("body > div:nth-child(1) > div:nth-child(2) > div > div > form > input").send_keys(Keys.ENTER)

In [44]:
driver.find_element_by_css_selector("body > div:nth-child(1) > div:nth-child(2) > div > div > form > input").send_keys(Keys.ENTER)

In [45]:
#artist > ul > li:nth-child(1) > div.media-body > div.media-heading > a

In [46]:
artist_id = driver.find_element_by_css_selector("#artist > ul > li > div.media-body > div.media-heading > a").get_attribute("href")[33:]

In [47]:
driver.get('https://chartmetric.io/artist?id=' + '200408')

In [ ]:
time.sleep(3)

In [55]:
sns = pd.DataFrame(columns=['artist', 'twitter', 'instagram', 'facebook', 'spotify', 'soundcloud', 'youtube'])

In [48]:
social = driver.find_element_by_id("socialStats")

In [70]:
twitter = int(social.find_element_by_id("twitterfollowers-total-fans").text.replace(',', ''))

In [71]:
instagram = int(social.find_element_by_id("instagram-total-fans").text.replace(',', ''))

In [72]:
facebook = int(social.find_element_by_id("facebooklikes-total-fans").text.replace(',', ''))

In [73]:
spotify = int(social.find_element_by_id("spotify-total-fans").text.replace(',', ''))

In [74]:
soundcloud = int(social.find_element_by_id("soundcloud-total-fans").text.replace(',', ''))

In [75]:
youtube = int(social.find_element_by_id("youtubesubscribers-total-fans").text.replace(',', ''))

In [76]:
sns.loc[len(sns)] = {"artist": artist, "twitter":twitter, "instagram":instagram, "facebook":facebook, 
                     "spotify":spotify, "soundcloud":soundcloud, "youtube":youtube,}    

In [77]:
sns

,artist,twitter,instagram,facebook,spotify,soundcloud,youtube
0,Cardi B,"2,987,228","23,162,441","5,255,066","2,168,550","187,516","3,089,104"
1,Cardi B,2987228,"23,162,441","5,255,066","2,168,550","187,516","3,089,104"
2,Cardi B,2987228,"23,162,441","5,255,066","2,168,550","187,516","3,089,104"
3,Cardi B,2987228,23162441,5255066,2168550,187516,3089104


In [ ]:
driver.close()

In [78]:
debut_df = pd.read_csv("../data/debut_album_1118.csv")

In [79]:
debut_df.shape[0]

1951

In [80]:
df_0_499 = debut_df[:500]

In [83]:
def get_followers(dataframe):
    
    ls = list(dataframe['artist'])
    sns_df = pd.DataFrame(columns=['artist', 'twitter', 'instagram', 'facebook', 'spotify', 'soundcloud', 'youtube'])
    
    # load webdriver and target webpage
    driver = webdriver.Chrome()
    driver.get('https://chartmetric.io/login')
    time.sleep(3)
    
    # load password for the website
    pickle_pw = pickle.load(open("chartmetric_pw.pickle", "rb"))

    # login
    driver.find_element_by_css_selector( "body > div > div:nth-child(2) > div.container.ng-scope > div > div > div > div > form > div:nth-child(4) > div > input" ).send_keys( "lucaseo1991@gmail.com" )
    driver.find_element_by_css_selector( "body > div > div:nth-child(2) > div.container.ng-scope > div > div > div > div > form > div:nth-child(5) > div > input" ).send_keys( pickle_pw )
    driver.find_element_by_css_selector("body > div > div:nth-child(2) > div.container.ng-scope > div > div > div > div > form > div:nth-child(7) > div > button").click()
    time.sleep(7)

    
    
    #input artist
    for artist in ls:
        
        # browse to search page
        driver.get('https://chartmetric.io/search')
        time.sleep(10)

        # input artist name
        driver.find_element_by_css_selector( "body > div:nth-child(1) > div:nth-child(2) > div > div > form > input" ).send_keys(artist)
        driver.find_element_by_css_selector("body > div:nth-child(1) > div:nth-child(2) > div > div > form > input").send_keys(Keys.ENTER)
        time.sleep(10)

        # enter artist info page
        artist_id = driver.find_element_by_css_selector("#artist > ul > li > div.media-body > div.media-heading > a").get_attribute("href")[33:]
        driver.get('https://chartmetric.io/artist?id=' + str(artist_id))
            
        time.sleep(10)
        
        social = driver.find_element_by_id("socialStats")

        time.sleep(5)
        
        try:
            twitter = int(social.find_element_by_id("twitterfollowers-total-fans").text.replace(',', ''))
        except ValueError:
            twitter = 0
        
        try:
            instagram = int(social.find_element_by_id("instagram-total-fans").text.replace(',', ''))
        except ValueError:
            instagram = 0
        
        try:
            facebook = int(social.find_element_by_id("facebooklikes-total-fans").text.replace(',', ''))
        except ValueError:
            soundcloud = 0
        
        try:
            spotify = int(social.find_element_by_id("spotify-total-fans").text.replace(',', ''))
        except ValueError:
            soundcloud = 0
        
        try:
            soundcloud = int(social.find_element_by_id("soundcloud-total-fans").text.replace(',', ''))
        except ValueError:
            soundcloud = 0
            
        try:
            youtube = int(social.find_element_by_id("youtubesubscribers-total-fans").text.replace(',', ''))
        except ValueError:
            soundcloud = 0

        
        data = {
            "artist": artist, 
            "twitter":twitter, 
            "instagram":instagram, 
            "facebook":facebook,
            "spotify":spotify, 
            "soundcloud":soundcloud,
            "youtube":youtube,
        }    
        
        sns_df.loc[len(sns_df)] = data
        
        time.sleep(2)
       
    driver.close()
        
    return sns_df
        






In [84]:
df_1 = get_followers(df_0_499)

ValueError: invalid literal for int() with base 10: ''

In [ ]:
print(df_1.shape)
df_1.head()

In [ ]:
driver = webdriver.Chrome()

In [ ]:
driver.get('https://chartmetric.io/login')

In [ ]:
time.sleep(5)

In [ ]:
pickle_pw = pickle.load(open("chartmetric_pw.pickle", "rb"))

In [ ]:
driver.find_element_by_css_selector( "body > div > div:nth-child(2) > div.container.ng-scope > div > div > div > div > form > div:nth-child(4) > div > input" ).send_keys( "lucaseo1991@gmail.com" )
driver.find_element_by_css_selector( "body > div > div:nth-child(2) > div.container.ng-scope > div > div > div > div > form > div:nth-child(5) > div > input" ).send_keys( pickle_pw )

In [ ]:
driver.find_element_by_css_selector("body > div > div:nth-child(2) > div.container.ng-scope > div > div > div > div > form > div:nth-child(7) > div > button").click()

In [ ]:
time.sleep(5)

In [ ]:
driver.get('https://chartmetric.io/search')

In [ ]:
time.sleep(5)

In [ ]:
artist = "Cardi B"

In [ ]:
driver.find_element_by_css_selector( "body > div:nth-child(1) > div:nth-child(2) > div > div > form > input" ).send_keys(artist)

In [ ]:
time.sleep(5)

In [ ]:
from selenium.webdriver.common.keys import Keys
driver.find_element_by_css_selector("body > div:nth-child(1) > div:nth-child(2) > div > div > form > input").send_keys(Keys.ENTER)

In [ ]:
driver.find_element_by_css_selector("#artist > ul > li > div.media-body > div.media-heading > a").text

In [ ]:
driver.find_element_by_link_text(artist).click()